In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/slerendu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import string
import nltk
nltk.download('wordnet')
from nltk import pos_tag, tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /home/slerendu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
data = pd.read_csv("data/training.1600000.processed.noemoticon.csv", sep=',',
                   encoding="ISO-8859-1", names=["target", "id", "date", "flag", "user", "text"])

### hide

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [6]:
model_file_name = "models/mod_bernoulli.h5"
loaded_model = pickle.load(open(model_file_name, 'rb'))

In [8]:
model_file_name_2 = "models/count_vectorizer.h5"
count_vectorizer = pickle.load(open(model_file_name_2, 'rb'))

### not hide

In [9]:
def clean_textism(sentence):
    neo_sentence = []
    for word in sentence:
        if word == 'u':
            neo_sentence.append('you')
        elif word == 'r':
            neo_sentence.append('are')
        elif word == 'ur':
            neo_sentence.append('your')
        elif word == 'some1':
            neo_sentence.append('someone')
        elif word == 'yrs':
            neo_sentence.append('years')
        elif word == 'hrs':
            neo_sentence.append('hours')
        elif word == 'mins':
            neo_sentence.append('minutes')
        elif word == 'secs':
            neo_sentence.append('seconds')
        elif word == 'pls' or word == 'plz':
            neo_sentence.append('please')
        elif word == '2morow':
            neo_sentence.append('tomorrow')
        elif word == '2day':
            neo_sentence.append('today')
        elif word == '2nite':
            neo_sentence.append('tonight')
        elif word == '4got' or word == '4gotten':
            neo_sentence.append('forget')
        elif word == 'amp' or word == 'quot' or word == 'lt' or word == 'gt' or word == '½25':
            neo_sentence.append('')
        else:
            neo_sentence.append(word)
    return neo_sentence

In [10]:
def get_wordnet_pos(word):
    """Return the corresponding character for a word use in the lemmatization
    
    Parameters:
    word (str): a word
    
    Returns:
    str: the corresponding character
    """
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

### Compute

In [36]:
phrase_neg = '@ home studying for maths wooot ! im so going to fail this shit '
phrase_pos = '@ashlili LIKE MEEEEEEEEE '

In [37]:
X = phrase_neg.split()
X_wo_arobas = [x for x in X if not x.startswith("@")]
X_new = [x for x in X_wo_arobas if not x.startswith("http")]
res = ' '.join(X_new)

In [38]:
words_to_exclude = [
    "...", ".."
]
tags_to_remove=["NNP", "VBG", "VBN", "CD"]

In [39]:
# building stopwords list
stopW = stopwords.words('english')
stopW.extend(string.punctuation)

temp_res = ""
# normalisation
temp_res = res.lower()
# tokenization
tk = tokenize.TweetTokenizer(reduce_len=True)
temp_res = tk.tokenize(temp_res)

# clean the sms language to usefull langage
temp_res = clean_textism(temp_res)

# remove stopwords
temp_res = [word for word in temp_res if word not in stopW]

# lemmatization
lemmatizer = WordNetLemmatizer()
temp_res = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in temp_res]

temp_res = pos_tag(temp_res)
temp_res = [x[0] for x in temp_res if x[1] not in tags_to_remove]

temp_res = [x for x in temp_res if x not in words_to_exclude]

res = ' '.join([x for x in temp_res])

In [40]:
res

'home study math wooot im go fail shit'

In [41]:
X_embed = count_vectorizer.transform([res])

In [42]:
X_embed

<1x800 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [43]:
temp_res = loaded_model.predict(X_embed)

In [44]:
temp_res

array([0])

In [45]:
if temp_res[0] > 0.5:
    print("Double Bam !!")
else:
    print("Small Bam......")

Small Bam......
